In [4]:
import tensorflow as tf
import cv2
import skimage.io
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
from keras import layers
from keras.layers import Input, Add, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,GlobalAveragePooling2D,Concatenate, ReLU, LeakyReLU,Reshape, Lambda
from keras.callbacks import ModelCheckpoint
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.preprocessing import image
from keras.utils import img_to_array
from imutils import paths
from keras import metrics
from tqdm import tqdm
import numpy as np
import keras.backend as K
import pickle
import random
import os

def identity_block(X, f, filters, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    return X

def convolutional_block(X, f, filters, stage, block, s = 2):
        
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    X_shortcut = X
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    return X

def ResNet50(input_shape = (128, 128, 3), classes = 6):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    X = AveragePooling2D(pool_size=(2, 2),name='avg_pool')(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    return model        


EPOCHS = 50
INIT_LR = 1e-4
BS = 64
IMAGE_DIMS = (128, 128, 3)

print("[INFO] loading images...")
data = []
labels = []
imagePaths = sorted(list(paths.list_images('dataset')))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)

	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

data = np.array(data, dtype = "float32")/255.0
labels = np.array(labels)
print("[INFO] datamatrix: {:.2f}MB".format(data.nbytes / (1024 * 1000)))
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

print("[INFO] serializing label binarizer...")
f = open('lb', "wb")
f.write(pickle.dumps(lb))
f.close()

# (trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = 0.3, random_state = 42)

# aug = ImageDataGenerator(rotation_range = 5, width_shift_range = 0.05, height_shift_range = 0.05, shear_range = 0.05, zoom_range = 0.05, horizontal_flip = True, fill_mode = "nearest")

print("[INFO] compiling model...")
#model = VGG16.build(width = IMAGE_DIMS[1], height = IMAGE_DIMS[0], depth = IMAGE_DIMS[2], classes = len(lb.classes_))
model = ResNet50(input_shape = IMAGE_DIMS  , classes = len(lb.classes_))
opt = Adam(learning_rate = INIT_LR,  decay = INIT_LR/50)
model.compile(loss = "categorical_crossentropy",optimizer = opt, metrics = ["accuracy"])

mcp_save = ModelCheckpoint('model.h5', save_best_only=True)

print("[INFO] training network...")
H = model.fit(data, labels,epochs = EPOCHS, batch_size= BS, validation_split=0.3,  callbacks=[mcp_save])



[INFO] loading images...
[INFO] datamatrix: 126.72MB
[INFO] serializing label binarizer...
[INFO] compiling model...
[INFO] training network...
Epoch 1/50
8/8 [==============================] - 126s 9s/step - loss: 2.2134 - accuracy: 0.2538 - val_loss: 1.7980 - val_accuracy: 0.1206
Epoch 2/50
8/8 [==============================] - 65s 8s/step - loss: 1.3160 - accuracy: 0.4751 - val_loss: 1.7990 - val_accuracy: 0.1809
Epoch 3/50
8/8 [==============================] - 68s 8s/step - loss: 1.0025 - accuracy: 0.5813 - val_loss: 1.8102 - val_accuracy: 0.1558
Epoch 4/50
8/8 [==============================] - 71s 9s/step - loss: 0.8376 - accuracy: 0.6898 - val_loss: 1.8393 - val_accuracy: 0.1809
Epoch 5/50
8/8 [==============================] - 69s 9s/step - loss: 0.6363 - accuracy: 0.7505 - val_loss: 1.8846 - val_accuracy: 0.1407
Epoch 6/50
8/8 [==============================] - 68s 9s/step - loss: 0.6323 - accuracy: 0.7527 - val_loss: 1.9354 - val_accuracy: 0.1809
Epoch 7/50
8/8 [===========

KeyboardInterrupt: 